In [1]:
!pip install transformers langchain langchain-huggingface

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 561.5/561.5 kB 11.2 MB/s eta 0:00:00a 0:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 443.5/443.5 kB 32.6 MB/s eta 0:00:00
  Attempting uninstall: huggingface-hub
    Found existing installation: huggingface-hub 0.33.1
    Uninstalling huggingface-hub-0.33.1:
      Successfully uninstalled huggingface-hub-0.33.1
  Attempting uninstall: langchain-core
    Found existing installation: langchain-core 0.3.66
    Uninstalling langchain-core-0.3.66:
      Successfully uninstalled langchain-core-0.3.66
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
datasets 3.6.0 requires fsspec[http]<=2025.3.0,>=2023.1.0, but you have fsspec 2025.5.1 which is incompatible.


# Question and Answer on summarized text
This is not anything really useful or properly functional, I just wanted to try the langchain.

In [2]:
from transformers import pipeline
from langchain_huggingface import HuggingFacePipeline
from langchain.prompts import PromptTemplate
# from transformers.utils.logging import set_verbosity_error    # for suppressing errors and warnings
from huggingface_hub import login

2025-08-11 07:40:54.118406: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:477] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1754898054.273128      36 cuda_dnn.cc:8310] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1754898054.317246      36 cuda_blas.cc:1418] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered


In [3]:
login()

## Pipelines

In [5]:
summarization_pipe = pipeline(
    task= 'summarization', 
    model= 'facebook/bart-large-cnn', 
    device= 0
)
summarizer = HuggingFacePipeline(pipeline= summarization_pipe)

Device set to use cuda:0


In [6]:
refinement_pipe = pipeline(
    task= 'summarization', 
    model= 'facebook/bart-large', 
    device= 0
)
refiner = HuggingFacePipeline(pipeline= refinement_pipe)

config.json: 0.00B [00:00, ?B/s]

pytorch_model.bin:   0%|          | 0.00/1.02G [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/1.02G [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/26.0 [00:00<?, ?B/s]

vocab.json: 0.00B [00:00, ?B/s]

merges.txt: 0.00B [00:00, ?B/s]

tokenizer.json: 0.00B [00:00, ?B/s]

Device set to use cuda:0


In [7]:
qa_pipeline = pipeline(
    task= 'question-answering',
    model= 'deepset/roberta-base-squad2',
    device= 1
)

config.json:   0%|          | 0.00/571 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/496M [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/79.0 [00:00<?, ?B/s]

vocab.json: 0.00B [00:00, ?B/s]

merges.txt: 0.00B [00:00, ?B/s]

special_tokens_map.json:   0%|          | 0.00/772 [00:00<?, ?B/s]

Device set to use cuda:1


## Summarization

In [11]:
summary_prompt = PromptTemplate.from_template('Summarize the following text in {word_limit} words: /n/n {text}')

In [12]:
summarization_chain = summary_prompt | summarizer | refiner

In [15]:
# taking user input
text_to_summarize = input('\nEnter text to summarize:\n')
length = input('\nEnter the length of summary (short/medium/long): ')

# maping the length to get word_limit
length_map = {
    'short': 100,
    'medium': 150,
    'long': 300
}
word_limit = length_map.get(length.lower(), 150)

# summarization
summary = summarization_chain.invoke({
    'word_limit': word_limit,
    'text': text_to_summarize
})
print(f'\n**Generated Summary:**\n{summary}')


Enter text to summarize:
 Artificial Intelligence (AI) has rapidly evolved from a niche research field into a transformative force reshaping industries, economies, and everyday life, powered by breakthroughs in machine learning, natural language processing, and computer vision, and fueled by unprecedented amounts of data and computational resources; from simple rule-based systems that could only perform narrowly defined tasks, AI has advanced to deep neural networks and large language models capable of understanding human language, generating creative content, detecting patterns in complex datasets, and even making autonomous decisions, enabling applications in healthcare where AI algorithms can diagnose diseases from medical images faster and sometimes more accurately than human doctors, in transportation where self-driving cars analyze real-time sensor inputs to navigate roads with minimal human intervention, in finance where predictive models assess market trends and manage risk, i

Your max_length is set to 128, but your input_length is only 63. Since this is a summarization task, where outputs shorter than the input are typically wanted, you might consider decreasing max_length manually, e.g. summarizer('...', max_length=31)



**Generated Summary:**
Artificial Intelligence (AI) has rapidly evolved from a niche research field into a transformative force reshaping industries, economies, and everyday life. The next decade will be a critical period in determining whether AI will serve as a tool for human flourishing or a source of disruption that outpaces our ability to adapt.


## Question Answer

In [17]:
while True:
    # ask question from user
    question = input('\nAsk a question about the summary (or type `/q` to stop):\n')

    # cheking the quit condition
    if question.lower() == '/q':
        break

    # inference
    qa_result = qa_pipeline(
        question= question,
        context= summary
    )

    print(f'\n**Answer**:\n{qa_result["answer"]}')


Ask a question about the summary (or type `/q` to stop):
 What is Artificial Intelligence



**Answer**:
a transformative force reshaping industries, economies, and everyday life



Ask a question about the summary (or type `/q` to stop):
 What exponentially increases the computation capabilities of AI



**Answer**:
Artificial Intelligence (AI) has rapidly evolved



Ask a question about the summary (or type `/q` to stop):
 how is it helpful



**Answer**:
a tool for human flourishing



Ask a question about the summary (or type `/q` to stop):
 predictive models



**Answer**:
disruption that outpaces our ability to adapt



Ask a question about the summary (or type `/q` to stop):
 /q
